In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_store = pd.read_csv('../input/store.csv', index_col=None)

In [ ]:
df_store.head()

In [ ]:
df_store.info()

In [ ]:
df_store.head()

In [ ]:
df_train = pd.read_csv('../input/train.csv', index_col=None, low_memory=False)

In [ ]:
print(df_train.shape)
df_train.head()

In [ ]:
df_train.info()

StateHoliday = '0' и 0 - это одно и то же, нужно исправить

In [ ]:
df_train.StateHoliday.unique()

In [ ]:
df_train.replace({'StateHoliday': {0: '0'}}, inplace=True)
df_train.StateHoliday.unique()

In [ ]:
df_train.DayOfWeek = df_train.DayOfWeek.astype(str)  # for dictvectorizer

In [ ]:
df = df_train[df_train.Open != 0].merge(df_store, on='Store').fillna(1)
df.drop(['Store', 'Date', 'Customers'], axis=1, inplace=True)

In [ ]:
df.shape, df.columns

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MinMaxScaler

dv = DictVectorizer()
mm = MinMaxScaler()

X_train = dv.fit_transform(df.drop(['Sales'],axis=1).fillna(0).to_dict('records'))

X_train = mm.fit_transform(X_train.toarray())

In [ ]:
y_train = df.Sales.values

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rgr = RandomForestRegressor(n_estimators=25, verbose=True, n_jobs=8)
rgr.fit(X_train, y_train)
print(rgr.score(X_train, y_train))

In [ ]:
df_test = pd.read_csv('../input/test.csv', index_col=None)

In [ ]:
print(df_test.shape)
df_test.head()

In [ ]:
df_test.replace({'StateHoliday': {0: '0'}}, inplace=True)
df_test.StateHoliday.unique()

In [ ]:
df_pred = df_test[df_test.Open != 0].merge(df_store, on='Store').fillna(1)
df_pred.drop(['Id', 'Store', 'Date'], axis=1, inplace=True)

In [ ]:
df_pred.shape

In [ ]:
X_test = dv.transform(df_pred.fillna(0).to_dict('records'))

X_test = mm.transform(X_test.toarray())

In [ ]:
X_test.shape

In [ ]:
rgr.predict(X_test)[:10]

In [ ]:
df_test.loc[df_test.Open != 0,'Sales'] = rgr.predict(X_test)
df_test.loc[df_test.Open == 0, 'Sales'] = 0

In [ ]:
df_test.head()

In [ ]:
out = pd.DataFrame({
    "Id": df_test.Id,
    "Sales": df_test.Sales.values
})
out.to_csv('submission.csv', index=False)